In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import json

In [ ]:
def fetch_bycode_with_retry(retries=5, delay=5):
    """Gửi request với retry khi lỗi."""
    for attempt in range(retries):
        try:
            url = "https://trade.ec.europa.eu/access-to-markets/api/v2/nomenclature/products"
            params = {
                "country": "AT",
                "lang": "EN"
            }

            response = requests.get(url, params=params, timeout=60)
            response.encoding = 'utf-8'
            if response.status_code == 200:
                return response
        except requests.RequestException as e:
            print(f"Request error: {e} - Retrying {attempt+1}/{retries}...")
            time.sleep(delay)
    return None

In [ ]:
def fetch_byid_with_retry(id, retries=5, delay=5):
    """Gửi request với retry khi lỗi."""
    for attempt in range(retries):
        try:
            url = "https://trade.ec.europa.eu/access-to-markets/api/v2/nomenclature/products"
            params = {
                "parent": id,
                "country": "AT",
                "lang": "EN"
            }

            response = requests.get(url,params=params,  timeout=60)
            response.encoding = 'utf-8'
            if response.status_code == 200:
                return response
        except requests.RequestException as e:
            print(f"Request error: {e} - Retrying {attempt+1}/{retries}...")
            time.sleep(delay)
    return None

In [16]:
section_info = fetch_bycode_with_retry().json()

In [18]:
section_df = pd.DataFrame(section_info)

In [19]:
section_df['index'] = section_df.index
section_df['crawled'] = ''


In [20]:
df = section_df.copy()
df

,id,code,description,hasChildren,section,index,crawled
0,-1,None,Live animals; animal products,True,"{'description': 'SECTION I', 'chapterFrom': '0...",0,
1,-2,None,Vegetable products,True,"{'description': 'SECTION II', 'chapterFrom': '...",1,
2,-3,None,Animal or vegetable fats and oils and their cl...,True,"{'description': 'SECTION III', 'chapterFrom': ...",2,
3,-4,None,"Prepared foodstuffs; beverages, spirits and vi...",True,"{'description': 'SECTION IV', 'chapterFrom': '...",3,
4,-5,None,Mineral products,True,"{'description': 'SECTION V', 'chapterFrom': '2...",4,
5,-6,None,Products of the chemical or allied industries,True,"{'description': 'SECTION VI', 'chapterFrom': '...",5,
6,-7,None,Plastics and articles thereof; rubber and arti...,True,"{'description': 'SECTION VII', 'chapterFrom': ...",6,
7,-8,None,"Raw hides and skins, leather, furskins and art...",True,"{'description': 'SECTION VIII', 'chapterFrom':...",7,
8,-9,None,Wood and articles of wood; wood charcoal; cork...,True,"{'description': 'SECTION IX', 'chapterFrom': '...",8,
9,-10,None,Pulp of wood or of other fibrous cellulosic ma...,True,"{'description': 'SECTION X', 'chapterFrom': '4...",9,


In [ ]:
import time
import pandas as pd

while ((df['hasChildren'] == True) & ((df['crawled'] == '') | pd.isna(df['crawled']) | (df['crawled'] == 'Error'))).any():
    for i in df.index:
        if df.loc[i, 'hasChildren'] and (df.loc[i, 'crawled'] == '' or pd.isna(df.loc[i, 'crawled']) or df.loc[i, 'crawled'] == 'Error'):
            id = df.loc[i, 'id']
            attempts = 0
            success = False
            error_occurred = False  # Biến để kiểm soát lỗi

            while attempts < 5 and not success:
                try:
                    print(f"Fetching index: {i} id: {id}... (Attempt {attempts}/5)")
                    
                    response = fetch_byid_with_retry(id)  
                    if response and response.status_code == 200:
                        df2 = pd.DataFrame(response.json())
                        df2['index'] = df2.index
                        df2['parent_id'] = id
                        print(df2)
                        
                        df = pd.concat([df, df2], ignore_index=True) 
                        df.loc[i, 'crawled'] = 'Done'
                        print(f"✅ Done fetching index: {i} id: {id}")
                        success = True  
                    else:
                        raise Exception(f"Failed request, status code: {response.status_code}")
                
                except Exception as e:
                    attempts += 1
                    error_message = str(e)
                    print(f"❌ Error fetching index: {i} id: {id}... (Attempt {attempts}/5) - {e}")
                    if attempts == 5: 
                        error_occurred = True

            if error_occurred: 
                df.loc[i, 'crawled'] = 'Error'
                df.loc[i, 'error_message'] = error_message
                print("⏳ Encountered multiple errors. Sleeping for 5 minutes...")
                time.sleep(300)  
                break  


Fetching index: 1022 id: 2085... (Attempt 0/5)
      id    code                                        description  \
0  16385  851610  Electric instantaneous or storage water heater...   
1  16485    None  Electric space-heating apparatus and electric ...   
2  16585    None  Electrothermic hairdressing or hand-drying app...   
3  16685  851640                           Electric smoothing irons   
4  44785  851650                                    Microwave ovens   
5  17185  851660  Other ovens; cookers, cooking plates, boiling ...   
6  17285    None                    Other electrothermic appliances   
7  17385  851680                         Electric heating resistors   
8  17485  851690                                              Parts   

   hasChildren intervalMin intervalMax selectable  index  parent_id  
0         True         NaN         NaN        NaN      0       2085  
1         True  8516210000  8516290000        NaN      1       2085  
2         True  8516310000  8516

In [23]:
df

,id,code,description,hasChildren,section,index,crawled,parent_id,selectable,intervalMin,intervalMax,error_message
0,-1,None,Live animals; animal products,True,"{'description': 'SECTION I', 'chapterFrom': '0...",0,Done,NaN,NaN,NaN,NaN,NaN
1,-2,None,Vegetable products,True,"{'description': 'SECTION II', 'chapterFrom': '...",1,Done,NaN,NaN,NaN,NaN,NaN
2,-3,None,Animal or vegetable fats and oils and their cl...,True,"{'description': 'SECTION III', 'chapterFrom': ...",2,Done,NaN,NaN,NaN,NaN,NaN
3,-4,None,"Prepared foodstuffs; beverages, spirits and vi...",True,"{'description': 'SECTION IV', 'chapterFrom': '...",3,Done,NaN,NaN,NaN,NaN,NaN
4,-5,None,Mineral products,True,"{'description': 'SECTION V', 'chapterFrom': '2...",4,Done,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2145,2337,None,"Other film, for colour photography (polychrome)",True,NaN,3,NaN,537.0,NaN,3702520000,3702560000,NaN
2146,2537,None,Other,True,NaN,4,NaN,537.0,NaN,3702960000,3702980000,NaN
2147,5237,370310,In rolls of a width exceeding 610 mm,False,NaN,0,NaN,637.0,True,NaN,NaN,NaN
2148,2937,370320,"Other, for colour photography (polychrome)",False,NaN,1,NaN,637.0,True,NaN,NaN,NaN


In [ ]:
df.to_csv('at_hs_info.csv', index=True)